# Imports

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from tensorflow import keras
from keras import models

from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *
from utilities.DecisionTree_BASIC import *

import utilities_LR

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sklearn

import matplotlib.pyplot as plt

# Config (2 I-Nets)

### Logistic Regression

In [2]:
config_LR = {
    'data': {
        'n_datasets': 9_000, # the number of datasets
        
        'n_samples': 4_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 8,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 3.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 42,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.3,
                'val_size': 0.1,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae', #tf.keras.losses.get(config['lambda_net']['loss_lambda']),
            'metrics': ['mae']
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 100,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.3,
                'val_size': 0.2,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 64,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'eval': {
        'n_datasets': 9_000,
        'n_samples_train': 2000,
        'n_samples_queryLambda': 1000, # _forLogRegBaseModel
        'n_samples_comparison': 1000 # compare inet and basemodel
    },
    'computation':{
        'n_jobs': 38,
        'use_gpu': False,
        'gpu_numbers': '31',
        'RANDOM_SEED': 1,   
    }
}

### Decision Tree

In [3]:

config_DT = {
    'function_family': {
        'maximum_depth': 3,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,    
        'dt_type': 'vanilla', #'SDT', 'vanilla'
    },
    'data': {
        'number_of_variables': 15, 
        'num_classes': 2,
        'categorical_indices': [],
        
        'use_distribution_list': True,
        'random_parameters_distribution': True, ##MAKEPATH DIFFERENT FILES
        'max_distributions_per_class': 1, # None; 0; int >= 1  
        'exclude_linearly_seperable': True,
        'data_generation_filtering': False,
        'fixed_class_probability': False,
        'balanced_data': True,
        'weighted_data_generation': False,
        'shift_distrib': False,
        
        'dt_type_train': 'vanilla', # (None, 'vanilla', 'SDT')
        'maximum_depth_train': 3, #None or int
        'decision_sparsity_train': 1, #None or int
        
        'function_generation_type': 'distribution',# 'make_classification_distribution', 'make_classification_distribution_trained', 'distribution', 'distribution_trained', 'make_classification', 'make_classification_trained', 'random_decision_tree', 'random_decision_tree_trained'
        'distrib_by_feature': True,
        'distribution_list': ['uniform', 'normal', 'gamma', 'beta', 'poisson'],#['uniform', 'gamma', 'poisson', 'exponential', 'weibull'],#['uniform', 'normal', 'gamma', 'exponential', 'beta', 'binomial', 'poisson'], 
        'distribution_list_eval': ['uniform', 'normal', 'gamma', 'beta', 'poisson'],#['uniform', 'gamma', 'poisson', 'exponential', 'weibull'],#['uniform', 'normal', 'gamma', 'beta', 'poisson'],
        
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 100,
        
        'noise_injected_level': 0, 
        'noise_injected_type': 'flip_percentage', # '' 'normal' 'uniform' 'normal_range' 'uniform_range'
        
        'data_noise': 0, #None or float
        
        'distrib_param_max': 5,
    }, 
    'lambda_net': {
        'epochs_lambda': 1000,
        'early_stopping_lambda': True, 
        'early_stopping_min_delta_lambda': 1e-3,
        'restore_best_weights': True,
        'patience_lambda': 50,
        
        'batch_lambda': 64,
        'dropout_lambda': 0,
        'lambda_network_layers': [128],
        'use_batchnorm_lambda': False,
        
        'optimizer_lambda': 'adam',
        'loss_lambda': 'binary_crossentropy', #categorical_crossentropy
        
        'number_of_lambda_weights': None,
        
        'number_initializations_lambda': 1, 
        
        'number_of_trained_lambda_nets': 100,
    },     
    
    'i_net': {
        #'dense_layers': [1024, 1024, 256, 2048, 2048],
        'dense_layers': [1792, 512, 512],
        #'dense_layers': [1792, 512, 512],
        
        #'dropout': [0, 0, 0, 0, 0.3],#[0.3, 0.3, 0.3, 0.3, 0.3],
        'dropout': [0, 0, 0.5],
        #'dropout': [0, 0, 0.5],

        #'hidden_activation': 'relu',
        'hidden_activation': 'sigmoid',
        #'hidden_activation': 'swish',

        #'optimizer': 'rmsprop', 
        'optimizer': 'adam', 
        #'optimizer': 'adam', 
        
        #'learning_rate': 0.001,
        'learning_rate': 0.001,
        #'learning_rate': 0.001, 
        
        'separate_weight_bias': False,
        
        'convolution_layers': None,
        'lstm_layers': None,        
        'additional_hidden': False,
        
        'loss': 'binary_crossentropy', #mse; binary_crossentropy; 'binary_accuracy'
        'metrics': ['binary_accuracy'], #soft_ or _penalized
        
        'epochs': 500, 
        'early_stopping': True,
        'batch_size': 256,

        'interpretation_dataset_size': 10000,
                
        'test_size': 5, #Float for fraction, Int for number 0
        'evaluate_distribution': True,
        'force_evaluate_real_world': False,
        
        'function_representation_type': 5, # 1=standard representation; 2=sparse representation with classification for variables; 3=softmax to select classes (n top probabilities)
        'normalize_lambda_nets': False,

        'optimize_decision_function': True, #False
        'function_value_loss': True, #False
                      
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2,3) #3=autoencoder dimensionality reduction
        
        'resampling_strategy': None,#'ADASYN', #'SMOTE', None
        'resampling_threshold': 0.25,#0.2,
        
        'nas': False,
        'nas_type': 'SEQUENTIAL', #options:(None, 'SEQUENTIAL', 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 60,
        'nas_optimizer': 'greedy' #'hyperband',#"bayesian",'greedy', 'random'
    },    
    
    'evaluation': {   
        #'inet_holdout_seed_evaluation': False,
        
        'number_of_random_evaluations_per_distribution': 10,
        'random_evaluation_dataset_size_per_distribution': 10_000, 
        'optimize_sampling': True,
            
        'random_evaluation_dataset_size': 500, 
        'random_evaluation_dataset_distribution': 'uniform', 
        
        'per_network_optimization_dataset_size': 5000,

        #'sklearn_dt_benchmark': False,
        #'sdt_benchmark': False,
        
        'different_eval_data': False,
        
        'eval_data_description': {
            ######### data #########
            'eval_data_function_generation_type': 'make_classification',
            'eval_data_lambda_dataset_size': 5000, #number of samples per function
            'eval_data_noise_injected_level': 0, 
            'eval_data_noise_injected_type': 'flip_percentage', # '' 'normal' 'uniform' 'normal_range' 'uniform_range'     
            ######### lambda_net #########
            'eval_data_number_of_trained_lambda_nets': 100,
            ######### i_net #########
            'eval_data_interpretation_dataset_size': 100,
        }
        
    },    
    
    'computation':{
        'load_model': False,
        'n_jobs': 15,
        'use_gpu': False,
        'gpu_numbers': '2',
        'RANDOM_SEED': 42,   
        'verbosity': 0
    }
}


In [4]:
computation_config = {
        'n_jobs': 38,
        'use_gpu': True,
        'gpu_numbers': '3',
        'RANDOM_SEED': 42,   
        'verbosity': 0
    }

# Settings

In [5]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = computation_config['gpu_numbers'] if computation_config['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if computation_config['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if computation_config['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if computation_config['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [7]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  0


In [8]:
random_model = generate_base_model(config_DT)#generate_base_model(config_DT, disable_batchnorm=True)

np.random.seed(config_DT['computation']['RANDOM_SEED'])
        
random_network_parameters = random_model.get_weights()
network_parameters_structure = [network_parameter.shape for network_parameter in random_network_parameters]  


try:
    use_distribution_list = config_DT['data']['use_distribution_list'] if config_DT['data']['max_distributions_per_class'] is not None else False
except:
    use_distribution_list = False if config_DT['data']['max_distributions_per_class'] is None else True

metrics = []
loss_function = None

if config_DT['i_net']['function_value_loss']:
    if config_DT['i_net']['function_representation_type'] == 1:
        pass
        #metrics.append(tf.keras.losses.get('mae'))
    if config_DT['i_net']['optimize_decision_function']:
        loss_function = inet_decision_function_fv_loss_wrapper(random_model, network_parameters_structure, config_DT, use_distribution_list=use_distribution_list)
        #metrics.append(inet_target_function_fv_loss_wrapper(config_DT))
        for metric in config_DT['i_net']['metrics']:
            metrics.append(inet_decision_function_fv_metric_wrapper(random_model, network_parameters_structure, config_DT, metric, use_distribution_list=use_distribution_list))  
            #metrics.append(inet_target_function_fv_metric_wrapper(config_DT, metric))  
    else:
        loss_function = inet_target_function_fv_loss_wrapper(config_DT)
        metrics.append(inet_decision_function_fv_loss_wrapper(random_model, network_parameters_structure, config_DT, use_distribution_list=use_distribution_list))
        for metric in config_DT['i_net']['metrics']:
            metrics.append(inet_target_function_fv_metric_wrapper(config_DT, metric))  
            metrics.append(inet_decision_function_fv_metric_wrapper(random_model, network_parameters_structure, config_DT, metric, use_distribution_list=use_distribution_list))  
else:
    if config_DT['i_net']['function_representation_type'] >= 3:
        if config_DT['i_net']['optimize_decision_function']:
            
            loss_function = inet_decision_function_fv_loss_wrapper_parameters(config_DT)
            
            metrics.append(inet_decision_function_fv_loss_wrapper(random_model, network_parameters_structure, config_DT, use_distribution_list=use_distribution_list))
            for metric in config_DT['i_net']['metrics']:
                metrics.append(inet_decision_function_fv_metric_wrapper(random_model, network_parameters_structure, config_DT, metric, use_distribution_list=use_distribution_list))    
            if False:
                metrics.append(inet_decision_function_fv_loss_wrapper(random_model, network_parameters_structure, config_DT, use_distribution_list=use_distribution_list))
                #metrics.append(inet_target_function_fv_loss_wrapper(config_DT))
                for metric in config_DT['i_net']['metrics']:
                    metrics.append(inet_decision_function_fv_metric_wrapper(random_model, network_parameters_structure, config_DT, metric, use_distribution_list=use_distribution_list))  
                    #metrics.append(inet_target_function_fv_metric_wrapper(config_DT, metric))                  
    else:
        raise SystemExit('Coefficient Loss not implemented for config_DTuration')
    
    if False:
        metrics.append(inet_target_function_fv_loss_wrapper(config_DT))
        metrics.append(inet_decision_function_fv_loss_wrapper(random_model, network_parameters_structure, config_DT, use_distribution_list=use_distribution_list))
        if config_DT['i_net']['optimize_decision_function']:
            raise SystemExit('Coefficient Loss not implemented for decision function optimization')            
        else:
            if config_DT['i_net']['function_representation_type'] == 1:
                loss_function = tf.keras.losses.get('mae') #inet_coefficient_loss_wrapper(inet_loss)
            else:
                raise SystemExit('Coefficient Loss not implemented for selected function representation')

                
# dill.dumps(loss_function)
# dill.dumps(metrics)

# Load Models

In [9]:
@tf.function
def custom_loss(y_coef_dataset_index_true, y_coef_pred):
    
    #print("true", type(y_coef_dataset_index_true))
    #print("true", y_coef_dataset_index_true)
    #
    #print("pred", type(y_coef_pred))
    #print("pred", y_coef_pred)
    #
    #print(y_coef_dataset_index_true.flatten().shape)
    #y_coef_dataset_index_true = y_coef_dataset_index_true.reshape([config['data']['n_targets']+1])
    index = y_coef_dataset_index_true[:, 0]
    y_coef_true = y_coef_dataset_index_true[:, 1:]
    
    #print("true", type(y_coef_true))
    #print("true", y_coef_true)
    
    index = tf.cast(index, tf.int32)
    #tf.print(index, output_stream=sys.stderr)
    
    valid_feature_data_sample = tf.gather(valid_feature_data, index)
    
    #y_true = tf.round(tf.matmul(valid_feature_data_sample, y_coef_true))
    #y_pred = tf.round(tf.matmul(valid_feature_data_sample, y_coef_pred))
    
    y_true = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_true))
    y_pred = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_pred))
    
    #tf.print(y_true, output_stream=sys.stderr)
    #tf.print(y_pred, output_stream=sys.stderr)

    #metric = tf.keras.metrics.binary_crossentropy(y_true, y_pred, from_logits=True, label_smoothing=0.0, axis=-1)
    metric = tf.keras.losses.BinaryCrossentropy(
                                from_logits=True,
                                label_smoothing=0.0,
                                axis=-1,
                                reduction='auto',
                                name='binary_crossentropy')
    val = metric(y_true, y_pred)
    #tf.print(val, output_stream=sys.stderr)
    return val

In [10]:
def load_LR_inet():
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = keras.models.load_model(path + '/modelKeras', custom_objects={'custom_loss': custom_loss})
    print(path)
    return model

In [11]:
def load_DT_inet_custom():
    
    loss_function_local = dill.dumps(loss_function)
    metrics_local = dill.dumps(metrics)
    
    path = './data/saved_models/lNetSize5000_numLNets100_var15_class2_distribution_xMax1_xMin0_xDistuniform_dNoise0_randParamDist_maxDistClass1_distribParamMax5_randClassProb_exLinSepun-no-ga-be-po_depth3_beta1_decisionSpars1_vanilla_fullyGrown/128_e1000ES0.001_b64_drop0_adam_binary_crossentropy_fixedInit1-seed42/inet_dense1792-512-512_drop0-0-0.5e500b256_adam_funcRep5_reshapeNone_depth3_beta1_decisionSpars1_vanilla_reshapeNone'    
    model = []
    from tensorflow.keras.utils import CustomObjectScope
    loss_function_local = dill.loads(loss_function_local)
    metrics_local = dill.loads(metrics_local)       

    #with CustomObjectScope({'custom_loss': loss_function}):
    custom_object_dict = {}
    custom_object_dict[loss_function.__name__] = loss_function_local
    for metric in  metrics_local:
        custom_object_dict[metric.__name__] = metrics_local        
        
    model = tf.keras.models.load_model(path, custom_objects=custom_object_dict) # #, compile=False
        
    return model


In [12]:
model_DT = load_DT_inet_custom()

In [13]:
model_LR = load_LR_inet()

KeyboardInterrupt: 

In [ ]:
model_LR.summary()

# Load Testing Data

### Logistic Regression

In [ ]:
#directory = utilities_LR.data_path_LR(config_LR)
#
## with open(directory + '/coef_list_LR_targetForInet.npy', "rb") as f:
#y_coef_truth_test_data_LR = np.load(directory + '/coef_list_targetForInet.npy', allow_pickle=True)
#

#y_coef_truth_test_data_LR =  np.load(utilities_LR.lambda_path_LR(config_LR) + '/lambda_generated_coef_list_target_for_inet.npy') ## get coef from lambda

In [ ]:
#directory = utilities_LR.lambda_path_LR(config_LR)
#
##with open(directory + '/coef_list_LR_targetForInet.npy', "rb") as f:
#x_lambda_weights_test_data_LR = np.load(directory + '/lambda_weights_list.npy', allow_pickle=True)

In [ ]:
#X_datasets_list_LR = np.zeros([config_LR['data']['n_datasets'], config_LR['data']['n_samples'], config_LR['data']['n_features']])
#
#if  config_LR['data']['n_targets'] < 2:
#    y_datasets_list_LR = np.zeros([config_LR['data']['n_datasets'], config_LR['data']['n_samples'], ])
#    coef_list_LR = np.zeros([config_LR['data']['n_datasets'], config_LR['data']['n_features'], ])
#else:
#    y_datasets_list_LR = np.zeros([config_LR['data']['n_datasets'], config_LR['data']['n_samples'], config_LR['data']['n_targets']])
#    coef_list_LR = np.zeros([config_LR['data']['n_datasets'], config_LR['data']['n_features'], config_LR['data']['n_targets']])

In [ ]:
directory = utilities_LR.data_path_LR(config_LR)

with open(directory + '/X_datasets_list_dataForLambda.npy', "rb") as f:
    X_datasets_list_LR_test = np.load(f, allow_pickle=True)
with open(directory + '/y_datasets_list_dataForLambda.npy', "rb") as f:
    y_datasets_list_LR_test = np.load(f, allow_pickle=True)

### Decision Trees

In [ ]:
# globals().update(generate_paths(config_DT, path_type='lambda_net'))

# directory = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'

#directory = './data/saved_function_lists/functions_lNetSize5000_numDatasets100_var15_class2_distribution_xMax1_xMin0_xDistuniform_dNoise0_randParamDist_maxDistClass1_distribParamMax5_randClassProb_exLinSepun-no-ga-be-po_depth3_beta1_decisionSpars1_vanilla_fullyGrown.csv'
#
#function_df = pd.read_csv(directory)

In [ ]:
# function_df.describe()

In [ ]:
# y_test_data_DT = 

# Evaluate Inet for LR

In [ ]:
def precision(tp, fp, tn, fn):
    return tp / (tp + fp)

In [ ]:
def recall(tp, fp, tn, fn):
    return tp / (tp + fn)

In [ ]:
def f1(tp, fp, tn, fn):
    pre = precision(tp, fp, tn, fn)
    rec = recall(tp, fp, tn, fn)
    return 2 * (pre * rec) / (pre + rec) 

# Evaluation on already known data

In [ ]:
X_test_weights = np.zeros([config_LR['eval']['n_datasets'], 8301, ])

X_queryLambda_test = np.zeros([config_LR['eval']['n_datasets'], config_LR['eval']['n_samples_queryLambda'], config_LR['data']['n_features']])

if  config_LR['data']['n_targets'] < 2:
    y_test_coefs = np.zeros([config_LR['eval']['n_datasets'], config_LR['data']['n_features'], ])
else:
    print("#################### NOT YET IMPLEMENTED ######################")

In [ ]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config_LR['lambda']['data_prep']['train_test_val_split']['test_size'] + config_LR['lambda']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config_LR['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config_LR['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config_LR['lambda']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config_LR['lambda']['data_prep']['train_test_val_split']['val_size'] / (config_LR['lambda']['data_prep']['train_test_val_split']['test_size'] + config_LR['lambda']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config_LR['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config_LR['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config_LR['lambda']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

In [ ]:
def get_LR(X, y):
    model = LogisticRegression(penalty='l2',
        dual=False,
        tol=0.0001,
        C=1.0,
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=None,
        random_state=None,
        solver='lbfgs',
        max_iter=100,
        multi_class='auto',
        verbose=0,
        warm_start=False,
        n_jobs=None,
        l1_ratio=None
                              )
    model.fit(X, y)
    return model

In [ ]:
def train_nn(X, y):
    # Data Prep
    X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                          y)
    
    # Model Def
    model = Sequential()
    model.add(BatchNormalization(input_dim=config_LR['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config_LR['data']['n_targets'], activation='sigmoid'))

    
    model.compile(optimizer=config_LR['lambda']['model_compile']['optimizer_lambda'],
                  loss=config_LR['lambda']['model_compile']['loss'],
                  metrics=config_LR['lambda']['model_compile']['metrics']
                 )
    
    # Model fit
    history = model.fit(x=X_train,
                        y=y_train,
                        batch_size=config_LR['lambda']['model_fit']['batch_size'],
                        epochs=config_LR['lambda']['model_fit']['epochs'],
                        verbose=config_LR['lambda']['model_fit']['verbose'],
                        callbacks=config_LR['lambda']['model_fit']['callbacks'],
                        validation_data=(X_val, y_val),
                        shuffle=config_LR['lambda']['model_fit']['shuffle'],
                        class_weight=config_LR['lambda']['model_fit']['class_weight'],
                        sample_weight=config_LR['lambda']['model_fit']['sample_weight'],
                        initial_epoch=config_LR['lambda']['model_fit']['initial_epoch'],
                        steps_per_epoch=config_LR['lambda']['model_fit']['steps_per_epoch'],
                        validation_steps=config_LR['lambda']['model_fit']['validation_steps'],
                        validation_batch_size=config_LR['lambda']['model_fit']['validation_batch_size'],
                        validation_freq=config_LR['lambda']['model_fit']['validation_freq'],
                       )
    
    
    
    return model

In [ ]:
def create_test_reg(i):
    
    ### TRAIN LAMBDA
    
    X = X_datasets_list_LR_test[i]
    y = y_datasets_list_LR_test[i]
    
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=config_LR['eval']['n_samples_queryLambda'] + config_LR['eval']['n_samples_comparison'])
    
    X_queryLambda, X_comparison, _, _ = train_test_split(X_temp, y_temp, test_size=config_LR['eval']['n_samples_comparison'])
    
    
    model_lambda = train_nn(X_train, y_train)
    
    
    ### GET COEF FROM LAMBDA
    
    y_pred = model_lambda.predict(X_queryLambda)
    
    y_pred = [1.0 if y>=0.5 else 0.0 for y in y_pred]
    
    log_reg_test = get_LR(X_queryLambda, y_pred)
    
    return np.concatenate([x.flatten() for x in model_lambda.get_weights()]), log_reg_test.coef_, X_queryLambda ## test_weights, test_coef
    

In [ ]:
parallel = Parallel(n_jobs=computation_config['n_jobs'], verbose=10, backend='loky') #loky
#parallel = Parallel(n_jobs=1, verbose=10, backend='loky') #loky

if config_LR['eval']['n_datasets'] > config_LR['data']['n_datasets']:
    print("Error: eval-n_datasets > data-n_datasets")

weights_coef = parallel(delayed(create_test_reg)(i) for i in range(config_LR['eval']['n_datasets']))
                                  
del parallel

In [ ]:
for i in range(config_LR['eval']['n_datasets']):
    X_test_weights[i] = weights_coef[i][0]
    y_test_coefs[i] = weights_coef[i][1]
    X_queryLambda_test[i] = weights_coef[i][2]

In [ ]:
def evaluateSingleSampleOnTestSet(i):
    X_test_weight = X_test_weights[i, :]
    y_test_coef = y_test_coefs[i, :]
    X_queryLambda_test_instance = X_queryLambda_test[i, :]
    
    
    X_test_weight = X_test_weight.reshape((1, 8301))
    
    y_test_coef_pred = model_LR.predict(x=X_test_weight,
        batch_size=None,
        verbose=0,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
                    )
    
    y_test_coef = y_test_coef.reshape([1, 20])
    
    model_truth = LogisticRegression()
    model_truth.coef_ = y_test_coef
    model_truth.intercept_ = 0
    model_truth.classes_ = np.array([0, 1])
    
    model_pred = LogisticRegression()
    model_pred.coef_ = y_test_coef_pred
    model_pred.intercept_ = 0
    model_pred.classes_ = np.array([0, 1])
    
    y_class_truth = model_truth.predict(X_queryLambda_test_instance)
    y_class_pred = model_pred.predict(X_queryLambda_test_instance)
    
    mse = sklearn.metrics.mean_squared_error(
        y_class_truth, y_class_pred
    )
    
    #score_groundTruthModel = model_groundTruth.score(X_datasets_list_LR[i], y_datasets_list_LR[i])
    #score_predModel = model_pred.score(X_datasets_list_LR[i], y_datasets_list_LR[i])
    #y_truth_set = model_groundTruth.predict(X_datasets_list_LR[i])
    #y_pred_set  = model_pred.predict(X_datasets_list_LR[i])
    tn, fp, fn, tp = confusion_matrix(y_class_truth, y_class_pred, labels=[1,0]).ravel()
    
    pre = precision(tp, fp, tn, fn)
    rec = recall(tp, fp, tn, fn)
    fone = f1(tp, fp, tn, fn)
    
    #results.append([i, score_groundTruthModel, score_predModel, mse, tp, fn, fp, tn, pre, rec, fone])
    
    return i+1, -1, -1, mse, tp, fn, fp, tn, pre, rec, fone

In [ ]:
#if os.path.isdir("/tmp") :
#    os.system('rm -R /tmp/*')
#
#os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp'

parallel = Parallel(n_jobs=computation_config['n_jobs'], 
                    verbose=10, backend='loky', 
                    timeout=None,
                    pre_dispatch='2 * n_jobs',
                    batch_size='auto',
                    temp_folder='/dev/shm',
                    max_nbytes='25M',
                    mmap_mode='r',
                    prefer=None,
                    require=None) #loky
#parallel = Parallel(n_jobs=1, verbose=10, backend='loky') #loky

result_list = parallel(delayed(evaluateSingleSampleOnTestSet)(i) for i in range(config_LR['eval']['n_datasets']))
                   
    
results = pd.DataFrame(columns=["index_0=aggregated", "scoreOnClassfication_BaseModel", "scoreOnClassfication_PredictedModel" , "mse",  "tp", "fn", "fp", "tn", "precision", "recall", "f1"], data=result_list)


In [ ]:
def save_eval_res_test(df):
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = df.to_csv(path + '/evalRes_test.csv')
    print(path)

In [ ]:
save_eval_res_test(results)

# Evaluate on arbitrary Data

In [ ]:
X_valid_weights = np.zeros([config_LR['eval']['n_datasets'], 8301, ])

X_queryLambda = np.zeros([config_LR['eval']['n_datasets'], config_LR['eval']['n_samples_queryLambda'], config_LR['data']['n_features']])

if  config_LR['data']['n_targets'] < 2:
    y_valid_coefs = np.zeros([config_LR['eval']['n_datasets'], config_LR['data']['n_features'], ])
else:
    print("#################### NOT YET IMPLEMENTED ######################")

In [ ]:
def get_LR(X, y):
    model = LogisticRegression(penalty='l2',
        dual=False,
        tol=0.0001,
        C=1.0,
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=None,
        random_state=None,
        solver='lbfgs',
        max_iter=100,
        multi_class='auto',
        verbose=0,
        warm_start=False,
        n_jobs=None,
        l1_ratio=None
                              )
    model.fit(X, y)
    return model

In [ ]:
def train_test_val_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=config_LR['lambda']['data_prep']['train_test_val_split']['test_size'] + config_LR['lambda']['data_prep']['train_test_val_split']['val_size'], 
                                                        random_state=config_LR['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                        shuffle=config_LR['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                        stratify=config_LR['lambda']['data_prep']['train_test_val_split']['stratify'])
    X_test, X_val, y__test, y_val = train_test_split(X_test, 
                                                    y_test, 
                                                    test_size=config_LR['lambda']['data_prep']['train_test_val_split']['val_size'] / (config_LR['lambda']['data_prep']['train_test_val_split']['test_size'] + config_LR['lambda']['data_prep']['train_test_val_split']['val_size']), 
                                                    random_state=config_LR['lambda']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config_LR['lambda']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config_LR['lambda']['data_prep']['train_test_val_split']['stratify'])
    return X_train, X_test, X_val, y_train, y_test, y_val
    

In [ ]:
def train_nn(X, y):
    # Data Prep
    X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
                                                                          y)
    
    # Model Def
    model = Sequential()
    model.add(BatchNormalization(input_dim=config_LR['data']['n_features']))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(config_LR['data']['n_targets'], activation='sigmoid'))

    
    model.compile(optimizer=config_LR['lambda']['model_compile']['optimizer_lambda'],
                  loss=config_LR['lambda']['model_compile']['loss'],
                  metrics=config_LR['lambda']['model_compile']['metrics']
                 )
    
    # Model fit
    history = model.fit(x=X_train,
                        y=y_train,
                        batch_size=config_LR['lambda']['model_fit']['batch_size'],
                        epochs=config_LR['lambda']['model_fit']['epochs'],
                        verbose=config_LR['lambda']['model_fit']['verbose'],
                        callbacks=config_LR['lambda']['model_fit']['callbacks'],
                        validation_data=(X_val, y_val),
                        shuffle=config_LR['lambda']['model_fit']['shuffle'],
                        class_weight=config_LR['lambda']['model_fit']['class_weight'],
                        sample_weight=config_LR['lambda']['model_fit']['sample_weight'],
                        initial_epoch=config_LR['lambda']['model_fit']['initial_epoch'],
                        steps_per_epoch=config_LR['lambda']['model_fit']['steps_per_epoch'],
                        validation_steps=config_LR['lambda']['model_fit']['validation_steps'],
                        validation_batch_size=config_LR['lambda']['model_fit']['validation_batch_size'],
                        validation_freq=config_LR['lambda']['model_fit']['validation_freq'],
                       )
    
    
    
    return model

In [ ]:
def create_valid_reg():
    
    ### TRAIN LAMBDA
    
    X, y =  sklearn.datasets.make_classification(n_samples=config_LR['eval']['n_samples_train'] + config_LR['eval']['n_samples_queryLambda'] + config_LR['eval']['n_samples_comparison'], 
                                                                                         n_features=config_LR['data']['n_features'],
                                                                                         n_informative=config_LR['data']['n_informative'],
                                                                                         n_redundant=config_LR['data']['n_features']-config_LR['data']['n_informative'],
                                                                                         n_repeated=0,
                                                                                         n_classes=config_LR['data']['n_targets']+1, 
                                                                                         n_clusters_per_class=config_LR['data']['n_clusters_per_class'],
                                                                                         weights=None,
                                                                                         flip_y=config_LR['data']['noise'],
                                                                                         class_sep=config_LR['data']['class_sep'],
                                                                                         shuffle=config_LR['data']['shuffle'],
                                                                                         random_state=config_LR['data']['random_state'])
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=config_LR['eval']['n_samples_queryLambda'] + config_LR['eval']['n_samples_comparison'])
    
    X_queryLambda, X_comparison, _, _ = train_test_split(X_temp, y_temp, test_size=config_LR['eval']['n_samples_comparison'])
    
    
    model_lambda = train_nn(X_train, y_train)
    
    
    ### GET COEF FROM LAMBDA
    
    y_pred = model_lambda.predict(X_queryLambda)
    
    y_pred = [1.0 if y>=0.5 else 0.0 for y in y_pred]
    
    log_reg_test = get_LR(X_queryLambda, y_pred)
    
    return np.concatenate([x.flatten() for x in model_lambda.get_weights()]), log_reg_test.coef_, X_queryLambda ## test_weights, test_coef
    

In [ ]:
parallel = Parallel(n_jobs=computation_config['n_jobs'], verbose=10, backend='loky') #loky
#parallel = Parallel(n_jobs=1, verbose=10, backend='loky') #loky


weights_coef = parallel(delayed(create_valid_reg)() for i in range(config_LR['eval']['n_datasets']))
                                  
del parallel

In [ ]:
for i in range(config_LR['eval']['n_datasets']):
    X_valid_weights[i] = weights_coef[i][0]
    y_valid_coefs[i] = weights_coef[i][1]
    X_queryLambda[i] = weights_coef[i][2]

In [ ]:
X_valid_weights.shape

In [ ]:
y_valid_coefs.shape

In [ ]:
X_queryLambda.shape

In [ ]:
def evaluateSingleSampleOnValidSet(i):
    X_valid_weight = X_valid_weights[i, :]
    y_valid_coef = y_valid_coefs[i, :]
    X_queryLambda_instance = X_queryLambda[i, :]
    
    
    X_valid_weight = X_valid_weight.reshape((1, 8301))
    
    y_valid_coef_pred = model_LR.predict(x=X_valid_weight,
        batch_size=None,
        verbose=0,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
                    )
    
    y_valid_coef = y_valid_coef.reshape([1, 20])
    
    model_truth = LogisticRegression()
    model_truth.coef_ = y_valid_coef
    model_truth.intercept_ = 0
    model_truth.classes_ = np.array([0, 1])
    
    model_pred = LogisticRegression()
    model_pred.coef_ = y_valid_coef_pred
    model_pred.intercept_ = 0
    model_pred.classes_ = np.array([0, 1])
    
    y_class_truth = model_truth.predict(X_queryLambda_instance)
    y_class_pred = model_pred.predict(X_queryLambda_instance)
    
    mse = sklearn.metrics.mean_squared_error(
        y_class_truth, y_class_pred
    )
    
    #score_groundTruthModel = model_groundTruth.score(X_datasets_list_LR[i], y_datasets_list_LR[i])
    #score_predModel = model_pred.score(X_datasets_list_LR[i], y_datasets_list_LR[i])
    #y_truth_set = model_groundTruth.predict(X_datasets_list_LR[i])
    #y_pred_set  = model_pred.predict(X_datasets_list_LR[i])
    tn, fp, fn, tp = confusion_matrix(y_class_truth, y_class_pred, labels=[1,0]).ravel()
    
    pre = precision(tp, fp, tn, fn)
    rec = recall(tp, fp, tn, fn)
    fone = f1(tp, fp, tn, fn)
    
    #results.append([i, score_groundTruthModel, score_predModel, mse, tp, fn, fp, tn, pre, rec, fone])
    
    return i+1, -1, -1, mse, tp, fn, fp, tn, pre, rec, fone

In [ ]:
#parallel = Parallel(n_jobs=computation_config['n_jobs'], verbose=10, backend='loky') #loky
parallel = Parallel(n_jobs=1, verbose=10, backend='loky') #loky

result_list = parallel(delayed(evaluateSingleSampleOnValidSet)(i) for i in range(config_LR['eval']['n_datasets']))
                   
    
results = pd.DataFrame(columns=["index_0=aggregated", "scoreOnClassfication_BaseModel", "scoreOnClassfication_PredictedModel" , "mse",  "tp", "fn", "fp", "tn", "precision", "recall", "f1"], data=result_list)


In [ ]:
results

In [ ]:
aggragated = pd.DataFrame(results.mean(numeric_only=True)).transpose()

In [ ]:
aggragated

In [ ]:
aggragated.at[0, "index_0=aggregated"] = 0
results = pd.concat([aggragated, results], axis=0, ignore_index=True)

In [ ]:
def save_eval_res_valid(df):
    path = utilities_LR.inet_path_LR(config_LR)
    
    model = df.to_csv(path + '/evalRes_valid.csv')
    print(path)

In [ ]:
save_eval_res_valid(results)